In [ ]:
# %load ../first.cell
%load_ext autoreload
%autoreload 2

In [ ]:
model_fp32 = "./vgg16-12.onnx"

import torch
import torchvision.datasets as datasets
from torch.utils.data import Dataloader
val_dataset = datasets.CIFAR10("./data",
                             train=False,
                             download=True)
val_dataloader = Dataloader(
                     val_dataset,
                     batch_size=32, shuffle=False,
                     ping_memory=True)

In [ ]:
import deepcrunch

In [ ]:
quantized_model = deepcrunch.quantize(model_fp32, output_path="./vgg16_quantize_nc.onnx", val_dataset=val_dataset)

In [ ]:
# If calibration or accuracy aware needed
eval_func = # Some function that takes in a model and returns accuracy or metrics. Usually a validation function.

quantized_model = deepcrunch.quantize(model_fp32, output_path="./vgg16_quantize_nc.onnx", val_dataset=val_dataset, eval_func=eval_func)

In [ ]:
original_model_size = deepcrunch.performance.size_in_mb(
    "./vgg16-12.onnx", human_readable=True
)

quantized_model_size = deepcrunch.performance.size_in_mb(
    "./vgg16_quantize_nc.onnx", human_readable=True
)

print(f"Original model size: {original_model_size}")
print(f"Quantized model size: {quantized_model_size}")

In [ ]:
import torch
import onnx
import onnxruntime as ort

print(torch.__version__)

device_id = 0
print(ort.get_device()) 

In [ ]:
providers = [(
    "CUDAExecutionProvider", 
    { 
        "device_id": device_id,
        "cudnn_conv_algo_search": "HEURISTIC", 
        "do_copy_in_default_stream": True
    }
)]

opts = ort.SessionOptions()
opts.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
opts.enable_cpu_mem_arena = False
opts.enable_mem_pattern = False
opts.enable_mem_reuse = False

opts.intra_op_num_threads = 1 
opts.inter_op_num_threads = 1

sess = ort.InferenceSession(
    "vgg16_quantize_nc.onnx", 
    sess_options=opts, 
    providers=providers)

In [ ]:
from deepcrunch.performance.model_latency import do_inference

do_inference("vgg16-12.onnx", "VGG16-12", input_dim=(1, 3, 224, 224), providers=providers)
do_inference("vgg16_float16.onnx", "Float16", input_dim=(1, 3, 224, 224), providers=providers)